**import necessary libraries**

In [ ]:
import requests
from bs4 import BeautifulSoup as bs

#### Load the webpage 

In [39]:
url = "https://en.wikipedia.org/wiki/Toy_Story_3"
r=requests.get(url)
#create the beautifulsoup object 
soup = bs(r.content)

In [40]:
#see the html page 
# print(soup.prettify())

In [41]:
info_box = soup.find(class_= "infobox vevent")
info_rows=info_box.find_all("tr")
#for row in info_rows:
   # print(row.prettify())

In [5]:
type(info_rows)

bs4.element.ResultSet

#### get the all info of movie

In [52]:
def get_content_value(row_data):
    if row_data.find('li'):
        return [li.get_text(" " , strip = False).replace("\xa0" , " ") for li in row_data.find_all("li")]
    else:
        return row_data.get_text(" " , strip = False).replace("\xa0" , " ") 

movie_info = {}

for index,row in enumerate(info_rows):
    if index == 0 :
        movie_info['title'] = row.find("th").get_text(" " , strip = False)
    elif index == 1:
        continue
    else : 
        content_key = row.find("th").get_text("\n" ,strip = False).replace("\n" ,"  ") # or get_text(" " , strip = False)
        content_value  = get_content_value(row.find("td"))
        movie_info[content_key] = content_value
    
movie_info
    

{'title': 'Toy Story 3',
 'Directed by': 'Lee Unkrich',
 'Produced by': 'Darla K. Anderson',
 'Screenplay by': 'Michael Arndt',
 'Story by': ['John Lasseter', 'Andrew Stanton', 'Lee Unkrich'],
 'Starring': ['Tom Hanks',
  'Tim Allen',
  'Joan Cusack',
  'Don Rickles',
  'Wallace Shawn',
  'John Ratzenberger',
  'Estelle Harris',
  'Blake Clark',
  'Jeff Pidgeon',
  'Ned Beatty',
  'Michael Keaton',
  'Jodi Benson',
  'John Morris'],
 'Music by': 'Randy Newman',
 'Cinematography': ['Jeremy Lasky', 'Kim White'],
 'Edited by': 'Ken Schretzmann',
 'Production  company ': ['Walt Disney Pictures', 'Pixar Animation Studios'],
 'Distributed by': 'Walt Disney Studios Motion Pictures',
 'Release date': ['June 12, 2010  ( 2010-06-12 )  ( Taormina Film Fest )',
  'June 18, 2010  ( 2010-06-18 )  (United States)'],
 'Running time': '103 minutes [1]',
 'Country': 'United States',
 'Language': 'English',
 'Budget': '$200 million [1]',
 'Box office': '$1.067 billion [1]'}

In [53]:
len(movie_info)

17

#### get info box for all movies

In [54]:
r = requests.get("https://en.m.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films")
soup = bs(r.content)

In [55]:
movies = soup.select(".wikitable.sortable i a")


In [56]:
movies[0:10]
movies[0]['href']

'/wiki/Academy_Award_Review_of_Walt_Disney_Cartoons'

In [57]:
movies[0]['title']

'Academy Award Review of Walt Disney Cartoons'

In [58]:

def get_content_value(row_data):
    if row_data.find('li'):
        return [li.get_text(" " , strip = True).replace("\xa0" , " ") for li in row_data.find_all("li")]   
    elif row_data.find("br"):     # solution for the long sentences get converted into list 
        return [text for text in row_data.stripped_strings]
    else:
        return row_data.get_text(" " , strip =True).replace("\xa0" , " ") 
    
#cleaning the references / tags [0] [1]
def clean_tags (soup):
    for tags in soup.find_all(["sup" , "span"]):    # removing wrt 'superscript tag'
        tags.decompose()

        
def get_info_box(url):
    
    r=requests.get(url)  
    soup = bs(r.content)    #create the beautifulsoup object 
    
    clean_tags(soup)      #cleaning the references / tags [0] [1]
       

    info_box = soup.find(class_= "infobox vevent")  # common class_ for all movies 
    info_rows=info_box.find_all("tr")
    movie_info={}    # empty dict to stotre the movie info   
    for index,row in enumerate(info_rows):
        if index == 0 :
            movie_info['title'] = row.find("th").get_text(" " , strip = True)
        else: 
            header = row.find("th")  # some of the tables not have header 
            if header:
                content_key = row.find("th").get_text(" " , strip = True) # or get_text(" " , strip = Truee)
                content_value  = get_content_value(row.find("td"))
                movie_info[content_key] = content_value
            
    return movie_info

In [59]:
 get_info_box("https://en.m.wikipedia.org/wiki/One_Little_Indian_(film)")

{'title': 'One Little Indian',
 'Directed by': 'Bernard McEveety',
 'Produced by': 'Winston Hibler',
 'Written by': 'Harry Spalding',
 'Starring': ['James Garner',
  'Vera Miles',
  'Pat Hingle',
  'Morgan Woodward',
  'Jodie Foster'],
 'Music by': 'Jerry Goldsmith',
 'Cinematography': 'Charles F. Wheeler',
 'Edited by': 'Robert Stafford',
 'Production company': 'Walt Disney Productions',
 'Distributed by': 'Buena Vista Distribution',
 'Release date': ['June 20, 1973'],
 'Running time': '90 Minutes',
 'Country': 'United States',
 'Language': 'English',
 'Box office': '$2 million'}

In [61]:
movies_info_list = []   # extracted movies info 

r = requests.get("https://en.m.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films")
soup = bs(r.content)
movies = soup.select(".wikitable.sortable i a")

base_path = "https://en.m.wikipedia.org/"      # base path of wikipedia

for index , movie in enumerate(movies):
  
    try:
        relative_path = movie['href']            #extracted path 
        complte_url  = base_path + relative_path      # base_path + extracted path 
        title = movie['title']        
        movies_info_list . append(get_info_box(complte_url))  # extracting the  all info 
        
    except Exception as  e:
        print(movie.get_text())
        print(e)

Zorro the Avenger
'NoneType' object has no attribute 'find'
The Sign of Zorro
'NoneType' object has no attribute 'find'
True-Life Adventures
'NoneType' object has no attribute 'find_all'
Luca
'NoneType' object has no attribute 'find_all'


In [62]:
len(movies_info_list)

444

In [63]:
movies_info_list[100]

{'title': 'Scandalous John',
 'Directed by': 'Robert Butler',
 'Produced by': 'Bill Walsh',
 'Written by': 'Bill Walsh',
 'Starring': ['Brian Keith', 'Alfonso Arau', 'Michele Carey'],
 'Cinematography': 'Frank V. Phillips',
 'Edited by': 'Cotton Warburton',
 'Production company': 'Walt Disney Productions',
 'Distributed by': 'Buena Vista Distribution',
 'Release date': 'June 22, 1971',
 'Running time': '113 minutes',
 'Country': 'United States',
 'Language': 'English'}

#### save / reload movoe data 


In [44]:
# to save the data
import json

def save_data(title,data):
    with open (title ,"w" , encoding = "utf-8") as f:
        json.dump(data,f , ensure_ascii = False, indent = 4 )

In [45]:
#load data
def load_data(title):
     with open (title , encoding = "utf-8") as f:
            return json.load(f)
    

In [46]:
#save_data("disney_raw_data.json" , movies_info_list)

## data cleaning

#### subtasks

-  ~~try to filtered out exception ~~
- ~~running time to int ~~
-  ~~remove tags [0] [1]~~
- ~~solution for the long sentences ~~
- ~~convert running time to int~~
- make datetime object 
- Budget & box_office to int 


In [192]:
# save_data("disney_data_cleaned_1.json" ,movies_info_list )  # not_fullycleaned 

In [65]:
movies_info =load_data("disney_data_cleaned_1.json")

In [71]:
len(movies_info)

444

In [73]:
movies_info[0:1]

[{'title': 'Academy Award Review of',
  'Production company': 'Walt Disney Productions',
  'Release date': ['May 19, 1937'],
  'Running time': '41 minutes (74 minutes 1966 release)',
  'Country': 'United States',
  'Language': 'English',
  'Box office': '$45.472'}]

In [77]:
x= {"b":4}
x.get("a" ,"n/a")

'n/a'

In [87]:
print ([x.get('Running time' , np.nan) for x in movies_info])

['41 minutes (74 minutes 1966 release)', '83 minutes', '88 minutes', '126 minutes', '74 minutes', '64 minutes', '70 minutes', '42 minutes', '65 min.', '71 minutes', '75 minutes', '94 minutes', '73 minutes', '75 minutes', '82 minutes', '68 minutes', '74 minutes', '96 minutes', '75 minutes', '84 minutes', '77 minutes', '92 minutes', '69 minutes', '81 minutes', ['60 minutes (VHS version)', '71 minutes (original)'], '127 minutes', '92 minutes', '76 minutes', '75 minutes', '73 minutes', '85 minutes', '81 minutes', '70 minutes', '90 min.', '80 minutes', '75 minutes', '83 minutes', '83 minutes', '72 minutes', '97 minutes', '75 minutes', '104 minutes', '93 minutes', '105 minutes', '95 minutes', '97 minutes', '134 minutes', '69 minutes', '92 minutes', '126 minutes', '79 minutes', '97 minutes', '128 minutes', '74 minutes', '91 minutes', '105 minutes', '98 minutes', '130 minutes', '89 min.', '93 minutes', '67 minutes', '98 minutes', '100 minutes', '118 minutes', '103 Minutes', '110 minutes', '80 

## subtask:


#### Subtask_1: 

- strip running time & convert in to the int 

In [160]:
"80 minutes"
#list = movies_info

def minutes_to_int(running_time):
    
    if  isinstance(running_time , list ):
        return int (running_time[0].split( )[0])
    
    elif running_time == "N/a":
        return None
    else:
        return int (running_time.split( )[0])
    
# applying 
for movie in movies_info:
    movie['Running time in minutes(int)'] = minutes_to_int (movie.get( 'Running time' ,'N/a'))
    

In [162]:
print([x.get('Running time in minutes(int)') for x in movies_info])  # cleaned time data

[41, 83, 88, 126, 74, 64, 70, 42, 65, 71, 75, 94, 73, 75, 82, 68, 74, 96, 75, 84, 77, 92, 69, 81, 60, 127, 92, 76, 75, 73, 85, 81, 70, 90, 80, 75, 83, 83, 72, 97, 75, 104, 93, 105, 95, 97, 134, 69, 92, 126, 79, 97, 128, 74, 91, 105, 98, 130, 89, 93, 67, 98, 100, 118, 103, 110, 80, 79, 91, 91, 97, 118, 139, 92, 131, 87, 116, 93, 110, 110, 131, 101, 108, 84, 78, 75, 164, 106, 110, 99, 113, 108, 112, 93, 91, 93, 100, 100, 79, 96, 113, 89, 118, 92, 88, 92, 87, 93, 93, 93, 90, 83, 96, 88, 89, 91, 93, 92, 97, 100, 100, 89, 91, 112, 115, 95, 91, 95, 104, 74, 48, 77, 104, 128, 101, 94, 104, 90, 100, 88, 93, 98, 100, 112, 84, 98, 97, 114, 96, 100, 109, 83, 90, 107, 96, 103, 91, 95, 105, 113, 80, 101, 89, 74, 90, 89, 110, 74, 93, 84, 83, 69, 77, 107, 93, 88, 108, 84, 121, 89, 104, 90, 86, 84, 108, 107, 96, 98, 105, 108, 94, 106, 102, 88, 102, 102, 97, 111, 100, 96, 98, 78, 81, 108, 89, 99, 89, 81, 92, 100, 89, 79, 91, 101, 104, 103, 86, 105, 93, 92, 98, 95, 93, 87, 93, 87, 128, 86, 95, 114, 93, 

#### Subtask_2: 

- seperate the release date 

In [190]:
[x.get( 'Release date', "N/a") for x in movies_info][0:5]

[['May 19, 1937'],
 ['December 21, 1937 ( Carthay Circle Theatre , Los Angeles , CA , premiere)'],
 ['February 7, 1940 ( Center Theatre )', 'February 23, 1940 (United States)'],
 ['November 13, 1940'],
 ['June 20, 1941']]

In [188]:
# sspliting / extracting the date from the list
def split_date(release_date):
    if isinstance(release_date ,list):
        entry =  release_date[0] 
        entry  = entry.split("(")[0]
        return entry
        # return release_date[0]..split("(")[0]
    elif release_date == "N/a":
        return None 
    else:
        return release_date
    
#apply 
for movie in movies_info:
    movie['Release Date Edited'] = split_date(movie.get("Release date" , "N/a"))

In [262]:
 [x.get( 'Release Date Edited') for x in movies_info]

['May 19, 1937',
 'December 21, 1937 ',
 'February 7, 1940 ',
 'November 13, 1940',
 'June 20, 1941',
 'October 23, 1941 ',
 'August 9, 1942 ',
 'August 24, 1942 ',
 'July 17, 1943',
 'December 21, 1944 ',
 'April 20, 1946 ',
 'November 12, 1946 ',
 'September 27, 1947',
 'May 27, 1948',
 'November 29, 1948 ',
 'October 5, 1949',
 'February 15, 1950 ',
 'June 22, 1950 ',
 'July 26, 1951 ',
 'March 13, 1952 ',
 'February 5, 1953 ',
 'August 8, 1953 ',
 'November 10, 1953',
 '26 October 1953 ',
 'August 17, 1954',
 'December 23, 1954',
 'May 25, 1955',
 'June 22, 1955',
 'September 14, 1955',
 'December 22, 1955',
 'June 8, 1956',
 'July 18, 1956',
 'November 6, 1956',
 'December 20, 1956',
 'June 19, 1957',
 'August 28, 1957',
 'December 25, 1957',
 'July 8, 1958',
 'August 12, 1958',
 'December 25, 1958',
 'January 29, 1959',
 'March 19, 1959',
 'June 24, 1959 ',
 'November 10, 1959',
 'January 21, 1960 ( Sarasota, FL )',
 'February 24, 1960',
 'May 19, 1960',
 'August 10, 1960',
 'Nov

In [256]:
None in [x.get( 'Release Date Edited') for x in movies_info]

True

#### Subtask_3: 

- convert datetime to datetime object

In [283]:
# delete the raw data 

#### save file 

In [285]:
save_data("Disney Dataset full edited.json" , movies_info)

TypeError: Object of type datetime is not JSON serializable